In [1]:
import backtrader as bt
import backtrader.feeds as btfeeds
import pandas as pd
import datetime

from pathlib import Path
import sys
project_root = str(Path(sys.path[0]).parents[0].absolute())
project_root
if project_root not in sys.path:
    sys.path.append(project_root)
    
from pathlib import Path
data_path = Path('.') / 'data'

# Acquisition des données

In [2]:
data = pd.read_csv(
    data_path / 'ohlc' / 'XBTEUR_1440.csv',
    names=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'trades_count'],
)
data

,timestamp,open,high,low,close,volume,trades_count
0,1378771200,97.0,97.00,97.00,97.00,1.000000,1
1,1378857600,99.9,99.90,96.75,96.75,2.050000,7
2,1379030400,96.0,96.99,96.00,96.00,1.030878,6
3,1379116800,96.0,96.00,95.00,95.00,2.290950,3
4,1379203200,95.0,95.00,95.00,95.00,0.931576,5
...,...,...,...,...,...,...,...
2750,1616803200,46681.0,47900.00,45805.00,47341.20,3045.673413,36395
2751,1616889600,47319.3,47873.00,46356.60,47333.10,1827.939901,30007
2752,1616976000,47327.8,49491.50,46684.10,48877.60,4094.518602,52469
2753,1617062400,48877.6,50468.90,48482.60,50085.90,3545.815358,49171


In [3]:
data['datetime'] = data['timestamp'].apply(lambda x: datetime.datetime.fromtimestamp(x))
data

,timestamp,open,high,low,close,volume,trades_count,datetime
0,1378771200,97.0,97.00,97.00,97.00,1.000000,1,2013-09-10 02:00:00
1,1378857600,99.9,99.90,96.75,96.75,2.050000,7,2013-09-11 02:00:00
2,1379030400,96.0,96.99,96.00,96.00,1.030878,6,2013-09-13 02:00:00
3,1379116800,96.0,96.00,95.00,95.00,2.290950,3,2013-09-14 02:00:00
4,1379203200,95.0,95.00,95.00,95.00,0.931576,5,2013-09-15 02:00:00
...,...,...,...,...,...,...,...,...
2750,1616803200,46681.0,47900.00,45805.00,47341.20,3045.673413,36395,2021-03-27 01:00:00
2751,1616889600,47319.3,47873.00,46356.60,47333.10,1827.939901,30007,2021-03-28 01:00:00
2752,1616976000,47327.8,49491.50,46684.10,48877.60,4094.518602,52469,2021-03-29 02:00:00
2753,1617062400,48877.6,50468.90,48482.60,50085.90,3545.815358,49171,2021-03-30 02:00:00


In [4]:
data.loc[
    (data.datetime >= datetime.datetime(2020, 1, 1)) &
    (data.datetime < datetime.datetime(2021, 1, 1))
][['datetime', 'open', 'high', 'low', 'close', 'volume']].to_csv(data_path / 'csv' / 'BTCEUR_2020_day.csv', index=False)

In [5]:
str((data_path / 'csv' / 'BTCEUR_2020_day.csv').absolute())

'/home/pmasse/pyfi/data/csv/BTCEUR_2020_day.csv'

In [6]:
data = btfeeds.GenericCSVData(
    dataname='/home/pmasse/pyfi/data/csv/BTCEUR_2020_day.csv',
    fromdate=datetime.datetime(2000, 1, 1),
    todate=datetime.datetime(2000, 12, 31),
    nullvalue=0.0,
    dtformat=('%Y-%m-%d %H:%M:%S'),
    datetime=0,
    high=1,
    low=2,
    open=3,
    close=4,
    volume=5,
    openinterest=-1
)

In [12]:
class TestStrategy(bt.Strategy):
    params = (
        ('maperiod', 15),
    )

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None

        # Add a MovingAverageSimple indicator
        self.sma = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.maperiod)

        # Indicators for the plotting show
        bt.indicators.ExponentialMovingAverage(self.datas[0], period=25)
        bt.indicators.WeightedMovingAverage(self.datas[0], period=25,
                                            subplot=True)
        bt.indicators.StochasticSlow(self.datas[0])
        bt.indicators.MACDHisto(self.datas[0])
        rsi = bt.indicators.RSI(self.datas[0])
        bt.indicators.SmoothedMovingAverage(rsi, period=10)
        bt.indicators.ATR(self.datas[0], plot=False)

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        # Simply log the closing price of the series from the reference
#         self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # Not yet ... we MIGHT BUY if ...
            if self.dataclose[0] > self.sma[0]:

                # BUY, BUY, BUY!!! (with all possible default parameters)
#                 self.log('BUY CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()

        else:

            if self.dataclose[0] < self.sma[0]:
                # SELL, SELL, SELL!!! (with all possible default parameters)
#                 self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()


In [14]:
%matplotlib qt

# Create a cerebro entity
cerebro = bt.Cerebro()

# Add a strategy
cerebro.addstrategy(TestStrategy)

# Datas are in a subfolder of the samples. Need to find where the script is
# because it could have been called from anywhere
# modpath = os.path.dirname(os.path.abspath(sys.argv[0]))
# datapath = os.path.join(modpath, '../../datas/orcl-1995-2014.txt')

# Create a Data Feed
data = btfeeds.GenericCSVData(
    dataname='/home/pmasse/pyfi/data/csv/BTCEUR_2020_day.csv',

#     fromdate=datetime.datetime(2000, 1, 1),
#     todate=datetime.datetime(2000, 12, 31),

    nullvalue=0.0,

    dtformat=('%Y-%m-%d %H:%M:%S'),

    datetime=0,
    high=1,
    low=2,
    open=3,
    close=4,
    volume=5,
    openinterest=-1
)

# Add the Data Feed to Cerebro
cerebro.adddata(data)

# Set our desired cash start
cerebro.broker.setcash(1000.0)

# Add a FixedSize sizer according to the stake
cerebro.addsizer(bt.sizers.FixedSize, stake=10)

# Set the commission
cerebro.broker.setcommission(commission=0.0)

# Print out the starting conditions
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Run over everything
cerebro.run()

# Print out the final result
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Plot the result
cerebro.plot()

RuntimeError: Cannot activate multiple GUI eventloops

In [17]:
import matplotlib
matplotlib.use('tez')

ValueError: Unrecognized backend string 'tez': valid strings are ['GTK3Agg', 'GTK3Cairo', 'MacOSX', 'nbAgg', 'Qt4Agg', 'Qt4Cairo', 'Qt5Agg', 'Qt5Cairo', 'TkAgg', 'TkCairo', 'WebAgg', 'WX', 'WXAgg', 'WXCairo', 'agg', 'cairo', 'pdf', 'pgf', 'ps', 'svg', 'template']

In [ ]:
cerebro.plot()